In [ ]:
IS_KAGGLE_ENV = False

In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import time
import copy
import matplotlib.pyplot as plt
from PIL import Image
import zipfile
import shutil 
from tqdm import tqdm

In [2]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import v2
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision import models as torchvision_models
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnet101, ResNet101_Weights
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
from torchvision.models import vit_b_16, ViT_B_16_Weights

In [4]:
def fix_random_seeds(seed=12345):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    
fix_random_seeds()

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Device count = {torch.cuda.device_count()}') 
print(f'Device: {device}')

Device count = 4
Device: cuda:0


In [ ]:
def create_dir_structure(is_kaggle=IS_KAGGLE_ENV):
    if is_kaggle:
        if not os.path.exists(DATA_ROOT):
            # Input data files are available in the "/kaggle/input/" directory.
            with zipfile.ZipFile('/kaggle/input/platesv2/plates.zip', 'r') as zip_obj:
                # Extract all the contents of zip file in current directory
                zip_obj.extractall()
            
            indexes = np.random.choice(20, size=4, replace=False)
            file_names = [f'{idx:04}.jpg' for idx in indexes]
            print('Moving files from train to val dir:', file_names)
            
            val_path = os.path.join(DATA_ROOT, 'val')
            os.makedirs(val_path)
            train_path = os.path.join(DATA_ROOT, 'train')
            class_names = ['cleaned', 'dirty']
            for class_name in class_names:
                dest_val_dir  = os.path.join(val_path, class_name)
                src_train_dir = os.path.join(train_path, class_name)
                os.makedirs(dest_val_dir)
                for file_name in file_names:
                    shutil.move(os.path.join(src_train_dir, file_name), dest_val_dir)
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
create_dir_structure()